In [ ]:
import pandas as pd
import datetime as dt
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Consigna

Vamos a trabajar con una base de datos elaborada por [Sysarmy](https://sysarmy.com/es/) (una comunidad de profesionales en el área de sistemas) que contiene los resultados de una encuesta se sueldos realizada en 2020. La encuesta tiene algunso campos tipificados y otros campos de texto libre.

Queremos analizar los resultados de esta encuesta. Nos interesan especialmente aquellas respuestas vinculadas a profesionales que trabajan con datos y que poseen título universitario (para eliminar algunos casos con comportamiento atípico). Los objetivos de esta práctica serán:

1. Construir una variable que nos indique hace cuánto tiempo fue respondida la encuesta.

2. Encontrar el salario promedio para los trabajos vinculados a datos. Puede ser general o abierto por las distintas variantes.

3. Contabilizar cuantos lenguajes de programación maneja la persona.


In [ ]:
sueldos = pd.read_csv("https://raw.githubusercontent.com/eea-uba/EEA-2020/master/Fuentes/encuesta_sueldos_sysarmy_1s2020.csv")
sueldos.head()

,timestamp,me_identifico,tengo,estoy_trabajando_en,donde_estas_trabajando,anos_de_experiencia,anos_en_la_empresa_actual,anos_en_el_puesto_actual,gente_a_cargo,nivel_de_estudios_alcanzado,estado,carrera,universidad,realizaste_cursos_de_especializacion,contribuis_a_proyectos_open_source,programas_como_hobbie,trabajo_de,plataformas,lenguajes_de_programacion,frameworksherramientas_y_librerias,bases_de_datos,qa_testing,ides,que_so_usas_en_tu_laptoppc_para_trabajar,y_en_tu_celular,tenes_guardias,cuanto_cobras_por_guardia,porcentajebruto_o_neto,tipo_de_contrato,salario_mensual_bruto_en_tu_moneda_local,salario_mensual_neto_en_tu_moneda_local,que_tan_conforme_estas_con_tu_sueldo,como_crees_que_esta_tu_sueldo_con_respecto_al_ultimo_semestre,recibis_algun_tipo_de_bono,a_que_esta_atado_el_bono,tuviste_ajustes_por_inflacion_en_2019,de_que_fue_el_ajuste_total,en_que_mes_fue_el_ultimo_ajuste,sufriste_o_presenciaste_situaciones_de_violencia_laboral,orientacion_sexual,tenes_algun_tipo_de_discapacidad,sentis_que_esto_te_dificulto_el_conseguir_trabajo,a_que_eventos_de_tecnologia_asististe_en_el_ultimo_ano,sos_miembro_de_alguna_comunidad_it,cantidad_de_empleados,actividad_principal,la_recomendas_como_un_buen_lugar_para_trabajar,como_calificas_las_politicas_de_diversidad_e_inclusion,a_cuantos_kilometros_de_tu_casa_queda_la_oficina,beneficios_extra,habias_respondido_nuestra_encuesta_en_ediciones_anteriores,cuales_consideras_que_son_las_mejores_empresas_de_it_para_trabajar_en_este_momentoen_tu_ciudad,sueldo_dolarizado
0,1/31/2020 6:50:26,Hombre,32,Argentina,Ciudad Autónoma de Buenos Aires,10,3,1,0,Secundario,Completado,0,0,"Sí, de forma particular",Sí,No,0,Linux,"Bash/Shell, SQL",NaN,"MySQL, Oracle",NaN,"Nano, Notepad++",Windows,No tengo celular / no es Smartphone,No,0,Porcentaje de mi sueldo,Full-Time,45,40.0,2,2,No,No recibo bono,Dos,10,1,Jamás,Homosexual,NaN,NaN,NaN,NaN,10001+,Producto basado en Software,8,10,6,"Capacitaciones y/o cursos, Comidas pagas / sub...",No,cognizant,False
1,1/28/2020 9:27:48,Hombre,30,Argentina,Ciudad Autónoma de Buenos Aires,3,3,3,0,Terciario,Completado,Diseño Gráfico,antonio berni,"Sí, de forma particular, Sí, los pagó un emple...",No,Sí,Technical Support,Windows Server,"CSS, HTML, Javascript",Bootstrap,NaN,NaN,Visual Studio Code,Windows,Android,No,0,Porcentaje de mi sueldo,Full-Time,48000,40000.0,1,1,No,No recibo bono,Dos,10,7,Jamás,Heterosexual,NaN,No,NaN,NaN,51-100,Otras industrias,7,3,35,Capacitaciones y/o cursos,No,mercadolibre,False
2,1/29/2020 16:54:29,Mujer,40,Argentina,Ciudad Autónoma de Buenos Aires,15,3,3,1,Universitario,Completado,Derecho,UBA - Universidad de Buenos Aires,No,No,No,Abogada,NaN,NaN,NaN,NaN,NaN,NaN,Windows,Android,No,0,Bruto,Full-Time,42000,35000.0,3,2,No,No recibo bono,Dos,10,10,En mi trabajo actual,Heterosexual,NaN,No,NaN,NaN,11-50,Servicios / Consultoría de Software / Digital,8,8,30,"Abono de celular y/o Internet, Capacitaciones ...",No,NaN,False
3,2/1/2020 5:46:25,Mujer,36,Argentina,Ciudad Autónoma de Buenos Aires,10,3,3,0,Universitario,Completado,Derecho,UNC - Universidad Nacional de Córdoba,No,No,No,Abogado,Lex doctor,Ninguno,Ninguno,Ninguno,Ninguno,Ninguno,Windows,Android,No,0,Neto,Full-Time,75000,60000.0,3,3,No,No recibo bono,No,0,0,Jamás,Heterosexual,NaN,NaN,NaN,NaN,51-100,Otras industrias,7,5,4,Viáticos,No,NaN,False
4,2/1/2020 17:51:21,Hombre,29,Argentina,Ciudad Autónoma de Buenos Aires,10,2,2,0,Universitario,Completado,Licenciatura en Marketing,IUEAN - Instituto Universitario Escuela Argent...,No,No,No,Account Manager-Sales,"Springserve, Pulse Point, Improve Digital, Mob...",NaN,NaN,NaN,NaN,NaN,Windows,Android,No,0,Neto,Full-Time,40000,34000.0,1,1,Menos de un sueldo,Performance individual,Uno,24,5,Jamás,Heterosexual,NaN,NaN,NaN,NaN,11-50,Otras industrias,5,8,7,Ninguna de las anteriores,No,NaN,False


In [ ]:
# observamos faltantes y tipos de datos
sueldos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 53 columns):
 #   Column                                                                                          Non-Null Count  Dtype  
---  ------                                                                                          --------------  -----  
 0   timestamp                                                                                       5982 non-null   object 
 1   me_identifico                                                                                   5982 non-null   object 
 2   tengo                                                                                           5982 non-null   int64  
 3   estoy_trabajando_en                                                                             5982 non-null   object 
 4   donde_estas_trabajando                                                                          5982 non-null   object 
 5   anos_de_exper

In [ ]:
# Antes de comenzar, filtramos solo los registros con educacion universitaria
sueldos = sueldos[sueldos.nivel_de_estudios_alcanzado == 'Universitario']

## 1) Fecha de encuesta

Pista: la función **datetime.datetime.now()** nos devuelve fecha y hora actuales.

In [ ]:
sueldos.timestamp.head()

2    1/29/2020 16:54:29
3      2/1/2020 5:46:25
4     2/1/2020 17:51:21
5     2/1/2020 23:13:41
9    1/31/2020 15:35:31
Name: timestamp, dtype: object

In [ ]:
# convertimos la variable datestamp a date
sueldos['fecha_de_respuesta'] = pd.to_datetime(sueldos.timestamp, format="%m/%d/%Y %H:%M:%S")
sueldos.fecha_de_respuesta.head()

2   2020-01-29 16:54:29
3   2020-02-01 05:46:25
4   2020-02-01 17:51:21
5   2020-02-01 23:13:41
9   2020-01-31 15:35:31
Name: fecha_de_respuesta, dtype: datetime64[ns]

In [ ]:
# generamos la variable de antiguedad (diferencia con hoy)
today = dt.datetime.now()

sueldos['dias_desde_respuesta'] = (today - sueldos.fecha_de_respuesta)
sueldos.dias_desde_respuesta.head()

2   500 days 19:25:46.884654
3   498 days 06:33:50.884654
4   497 days 18:28:54.884654
5   497 days 13:06:34.884654
9   498 days 20:44:44.884654
Name: dias_desde_respuesta, dtype: timedelta64[ns]

## 2) Trabajo con datos

In [ ]:
sueldos.trabajo_de.unique()

array(['Abogada', 'Abogado', 'Account Manager-Sales', 'Adm',
       'Administración', 'Administrador de Infraestructura',
       'Administrativa', 'Administrativa contable',
       'Administrative Assistant', 'Administrativo',
       'Administrativo en farmacia', 'Analista',
       'Analista de Operaciones de Servicios',
       'Analista de prevención de lavado de activos',
       'Analista de Soporte', 'Analista Funciona ERP',
       'Functional Analyst', 'Developer', 'Analista SAP',
       'Analista Senior.', 'Analyst support', 'Anlytics Engineer',
       'Application Support', 'Applications Support Analyst', 'Architect',
       'Asistente de Sistemas', 'Assistant', 'Atención al cliente',
       'Atención al publico', 'Auditor de TI', 'Auditoria',
       'BI Analyst / Data Analyst', 'Data Scientist / Data Engineer',
       'DBA Especializado / Gurú', 'DBA', 'BI Sr. Manager',
       'Business Analyst', 'Cadete', 'Cajera', 'Cajero', 'Call center',
       'Captador de donaciones', 'Chef

In [ ]:
# llevamos todo a minuscula
sueldos['trabajo_de'] = sueldos.trabajo_de.str.lower()

In [ ]:
# creo una variable binaria para ver si trabaja con datos
sueldos['trabajo_con_datos'] = sueldos.trabajo_de.str.contains('data')

In [ ]:
# Verificamos nuestro filtro
sueldos[sueldos['trabajo_con_datos'] == True].trabajo_de.unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data entry', 'data entry and management',
       'lider data analyst qa - tester'], dtype=object)

In [ ]:
# Filtramos los profesionales de datos
sueldos_data = sueldos[sueldos['trabajo_con_datos'] == True]

In [ ]:
# Normalizamos el puesto de 'data entry' y 'data analyst
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace('data entry and management', 'data entry')
sueldos_data['trabajo_de'] = sueldos_data.trabajo_de.str.replace('lider data analyst qa - tester', 'bi analyst / data analyst')

In [ ]:
sueldos_data.trabajo_de.unique()

array(['bi analyst / data analyst', 'data scientist / data engineer',
       'data entry'], dtype=object)

In [ ]:
# Ahora vemos el salario promedio
np.mean(sueldos_data.salario_mensual_bruto_en_tu_moneda_local)

128527.74056603774

In [ ]:
sueldos_data.groupby('trabajo_de').agg({'salario_mensual_bruto_en_tu_moneda_local': [np.mean, np.median]})

salario_mensual_bruto_en_tu_moneda_local       
                                                                   mean median
trabajo_de                                                                    
bi analyst / data analyst                                 154215.290323  61250
data entry                                                 22000.000000  22000
data scientist / data engineer                             93967.267442  80000

## 3) Lenguajes de programación

In [ ]:
sueldos_data.lenguajes_de_programacion

240                                    Java, SQL, VBA
242                                               SQL
244     .NET, C#, CSS, HTML, PHP, Python, R, SQL, VBA
245                  Assembler, C#, COBOL, HTML, Java
247                                       Python, SQL
                            ...                      
904                                               NaN
906                     HTML, Javascript, Python, SQL
908                                               SQL
912                                               SQL
3704                          HTML, Java, Python, SQL
Name: lenguajes_de_programacion, Length: 212, dtype: object

In [ ]:
sueldos_data['lenguajes_de_programacion_split'] = sueldos_data.lenguajes_de_programacion.str.split(',')

In [ ]:
sueldos_data['cantidad_lenguajes'] = sueldos_data.lenguajes_de_programacion_split.str.len()

In [ ]:
sueldos_data.groupby('trabajo_de').agg({'cantidad_lenguajes': [np.nanmean, np.median]})

cantidad_lenguajes       
                                          nanmean median
trabajo_de                                              
bi analyst / data analyst                2.271845    2.0
data entry                                    NaN    NaN
data scientist / data engineer           3.059524    3.0